In [4]:
import pandas
import numpy

In [5]:
mabc = pandas.read_csv('data/monthly_arrivals_by_country_filtered.csv')

In [6]:
fsi = pandas.read_csv("data/fragilestatesindex_joined.csv")

In [7]:
PSQ_TMS = pandas.read_csv("data/PSQ_TMS.csv", skiprows=range(5), na_values="*")

In [8]:
test = PSQ_TMS[PSQ_TMS["Country Name"] == "Afghanistan"]

In [9]:
test.drop(["Origin / Returned from", "Population type"], axis=1,
    inplace=True)

/Users/steveellison/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [10]:
sums = test.iloc[:,1:].sum()

In [11]:
values = pandas.Series(["Afghanistan"], ["Country Name"]).append(sums)

In [12]:
test_df = pandas.DataFrame(values)

In [13]:
test_df.transpose()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,1051112,1226098,3376479,912292,1066779,911679,527710,535632,515659,361985,1318019,1548374,1499351,985191


In [14]:
psq_tms_clean = pandas.read_csv("data/PSQ_TMS_filtered.csv")

In [15]:
gdp_info_raw = pandas.read_csv("data/world_bank_gdp/GDP_2011-2015.csv",skiprows=4)

In [16]:
gdp_clean = pandas.read_csv("data/world_bank_gdp/GDP_filtered.csv")

In [17]:
pov_raw = pandas.read_csv("data/world_bank_poverty/Poverty_raw.csv", skiprows=4)

In [18]:
pov_clean = pandas.read_csv("data/world_bank_poverty/Poverty_filtered.csv")

In [19]:
joined_data = pandas.merge(psq_tms_clean[["2013", "Country Name"]], fsi, on="Country Name")

In [20]:
finance_data = pandas.read_csv("data/financial_data.csv")

In [21]:
commodity_data = pandas.read_csv("data/commodities_filtered.csv")

In [22]:
finance_data_filtered = finance_data[["Country", "WEO Subject Code"] +[str(x) for x in range(2000, 2014)]]

In [23]:
finance_data_filtered.replace("n/a", numpy.nan, inplace=True)

/Users/steveellison/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
finance_data_filtered = finance_data_filtered[finance_data["WEO Subject Code"] == "NGDPD"]

In [25]:
finance_data_filtered.drop("WEO Subject Code", axis=1, inplace=True)

In [26]:
finance_data_filtered.rename(columns={str(col): "GDP (Billions of USD), " + str(col) for col in range(2000, 2014)}, inplace=True)

In [27]:
finance_data_filtered.rename(columns={"Country": "Country Name"}, inplace=True)

In [28]:
joined_data = pandas.merge(joined_data, finance_data_filtered, on="Country Name")

In [29]:
joined_data.rename(columns={"2013" : "Total Refugee Count, 2013"}, inplace=True)

In [30]:
joined_data = joined_data.applymap(lambda x: str(x).replace(",",""))

In [31]:
np_joined = joined_data.drop(["Country Name"], axis=1).as_matrix().astype("float64")
np_joined = np_joined[~numpy.isnan(np_joined).any(axis=1)]

In [32]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVR

from sklearn import metrics

In [33]:
target = np_joined[:,0]
features = np_joined[:,1:]

ftrain, ftest, ttrain, ttest = train_test_split(features, target, test_size=0.2, random_state=41234)

In [34]:
linear_svr = LinearSVR(loss='epsilon_insensitive', epsilon=0)

In [35]:
linear_svr.fit(ftrain, ttrain)
preds_svr = linear_svr.predict(ftest)
print numpy.sqrt(metrics.mean_squared_error(ttest, preds_svr))
print metrics.r2_score(ttest, preds_svr)

261220.844675
-0.0467511016811


In [36]:
metadata = pandas.read_csv("data/cpia/CPIA-Series.csv")
metadata[0:5]

,Series Code,Topic,Indicator Name,Long definition,Periodicity,Aggregation method,Limitations and exceptions,Source,Statistical concept and methodology,Development relevance
0,IQ.CPA.HRES.XQ,Public Sector: Policy & institutions,CPIA building human resources rating (1=low to...,Building human resources assesses the national...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
1,IQ.CPA.BREG.XQ,Public Sector: Policy & institutions,CPIA business regulatory environment rating (1...,Business regulatory environment assesses the e...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
2,IQ.CPA.DEBT.XQ,Public Sector: Policy & institutions,CPIA debt policy rating (1=low to 6=high),Debt policy assesses whether the debt manageme...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
3,IQ.CPA.ECON.XQ,Public Sector: Policy & institutions,CPIA economic management cluster average (1=lo...,The economic management cluster includes macro...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
4,IQ.CPA.REVN.XQ,Public Sector: Policy & institutions,CPIA efficiency of revenue mobilization rating...,Efficiency of revenue mobilization assesses th...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...


In [37]:
series_codes = metadata["Series Code"]

In [38]:
data = pandas.read_csv("data/cpia/CPIA-Data.csv")
print data.shape
data[numpy.isnan(data["2006"])].shape

(1995, 14)


(150, 14)

In [39]:
import cleanup_cpia as cpia

cpia.cleanup_cpia()

cleanup_cpia.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  axis=1, inplace=True)
/Users/steveellison/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [40]:
cpia_clean = pandas.read_csv("data/cpia/CPIA-Data_clean.csv")

In [41]:
cpia_clean[0:5]

,Country Name,"IQ.CPA.HRES.XQ, 2005","IQ.CPA.HRES.XQ, 2006","IQ.CPA.HRES.XQ, 2007","IQ.CPA.HRES.XQ, 2008","IQ.CPA.HRES.XQ, 2009","IQ.CPA.HRES.XQ, 2010","IQ.CPA.HRES.XQ, 2011","IQ.CPA.HRES.XQ, 2012","IQ.CPA.HRES.XQ, 2013",...,"IQ.CPA.IRAI.XQ, 2005","IQ.CPA.IRAI.XQ, 2006","IQ.CPA.IRAI.XQ, 2007","IQ.CPA.IRAI.XQ, 2008","IQ.CPA.IRAI.XQ, 2009","IQ.CPA.IRAI.XQ, 2010","IQ.CPA.IRAI.XQ, 2011","IQ.CPA.IRAI.XQ, 2012","IQ.CPA.IRAI.XQ, 2013","IQ.CPA.IRAI.XQ, 2014"
0,Afghanistan,NaN,3,3,3,3,3,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
1,Afghanistan,NaN,3,3,3,3,3,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
2,Afghanistan,NaN,3,3,3,3,3,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
3,Afghanistan,NaN,3,3,3,3,3,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
4,Afghanistan,NaN,3,3,3,3,3,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65


In [42]:
target = np_joined[:,0]

In [43]:
joined_data

,"Total Refugee Count, 2013",Country Name,Demographic Pressures 2006,External Intervention 2006,Factionalized Elites 2006,Group Grievance 2006,Human Flight 2006,Human Rights 2006,Legitimacy of the State 2006,Poverty and Economic Decline 2006,...,"GDP (Billions of USD), 2004","GDP (Billions of USD), 2005","GDP (Billions of USD), 2006","GDP (Billions of USD), 2007","GDP (Billions of USD), 2008","GDP (Billions of USD), 2009","GDP (Billions of USD), 2010","GDP (Billions of USD), 2011","GDP (Billions of USD), 2012","GDP (Billions of USD), 2013"
0,985191.0,Afghanistan,7.9,10.0,8.0,9.1,7.0,8.2,8.3,7.5,...,5.146,6.167,6.925,8.556,10.297,12.066,15.325,17.89,20.296,20.17
1,95921.0,Algeria,6.0,5.8,6.4,7.1,5.6,7.5,7.5,3.5,...,85.327,103.198,117.027,134.977,171.001,137.054,161.207,199.394,209.005,209.703
2,45462.0,Angola,8.0,7.6,8.0,6.3,5.0,7.8,8.8,4.9,...,19.641,28.234,41.789,60.449,84.178,75.492,82.471,104.116,115.342,124.912
3,4164.0,Argentina,3.0,3.0,2.8,4.0,4.0,3.7,3.5,4.2,...,182.163,221.411,263.044,329.764,406.001,378.502,462.839,559.846,607.719,622.055
4,47878.0,Australia,2.5,1.0,1.5,3.0,1.0,2.5,1.0,1.5,...,655.722,733.963,780.824,946.377,1036.05,982.382,1245.98,1502.59,1558.68,1501.09
5,614260.0,Azerbaijan,6.0,7.0,7.5,7.3,5.0,6.0,8.1,5.9,...,8.682,13.245,21.027,33.09,46.378,44.289,52.913,64.819,68.7,73.537
6,231145.0,Bangladesh,9.0,6.0,8.9,9.5,8.5,7.8,9.0,7.0,...,68.593,70.921,75.77,85.604,97.062,108.896,122.039,131.079,141.705,161.297
7,7380.0,Belarus,9.0,8.0,8.0,5.5,3.5,7.3,9.0,6.3,...,23.142,30.21,36.962,45.276,60.752,49.209,55.221,59.735,63.615,73.098
8,39477.0,Belgium,3.0,2.5,1.5,3.5,1.0,1.5,1.5,2.0,...,371.388,388.029,410.173,472.475,521.084,485.781,484.45,527.491,498.055,521.565
9,291.0,Benin,6.0,6.7,3.8,4.0,6.9,4.8,6.5,6.5,...,4.527,4.812,5.147,5.977,7.166,7.116,6.983,7.821,8.122,9.114


In [44]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVR

from sklearn import metrics

In [54]:
labels = np_joined[:,0]
features = np_joined[:,1:]

from sklearn.cross_validation import train_test_split
ftrain, ftest, ltrain, ltest = train_test_split(features, labels, test_size=0.2, random_state=19890223)

In [75]:
from sklearn import linear_model

lossParam = ["hinge", "log", "modified_huber", "squared_hinge", "perceptron", "squared_loss", "huber", "epsilon_insensitive"]
penaltyParam = ["none", "l2", "l1", "elasticnet"]
matrixError = [[0 for x in range(len(penaltyParam))] for y in range(len(lossParam))] 
minError = float("inf")
minI = 0;
minJ = 0;
minR2 = float("inf")
minI2 = 0;
minj2 = 0;
matrixR2 = [[0 for x in range(len(penaltyParam))] for y in range(len(lossParam))] 

for k in range(0, 3):
    for i in range(0, len(lossParam)):
        for j in range(0, len(penaltyParam)):
            sgd = linear_model.SGDClassifier(learning_rate='constant', loss=lossParam[i], eta0=0.01, n_iter=10, penalty=penaltyParam[j], alpha=0.001)
            sgd.fit(ftrain,ltrain)
            prediction = sgd.predict(ftest)        
            matrixError[i][j] = (numpy.sqrt(metrics.mean_squared_error(ltest, prediction)))
            if matrixError[i][j] < minError:
                minError = matrixError[i][j]
                minI = i;
                minJ = j;
            matrixR2[i][j] = metrics.r2_score(ltest, prediction)
            if matrixR2[i][k] < minR2:
                minI2 = i;
                minJ2 = j;
                minR2 = matrixR2[i][j]
                
print "loss: " + lossParam[minI] + ", penalty: " + penaltyParam[minJ]
print matrixError[minI][minJ]            

print "loss: " + lossParam[minI2] + ", penalty: " + penaltyParam[minJ2]
print matrixR2[minI2][minJ2]


loss: modified_huber, penalty: elasticnet
179310.811869
loss: huber, penalty: l1
-447.176471798


In [81]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(max_depth=6)
dt.fit(ftrain, ltrain)
prediction = dt.predict(ftest)
err = numpy.sqrt(metrics.mean_squared_error(ltest, prediction))
r2 = metrics.r2_score(ltest, prediction)

print "basic decision tree:"
print "error: "  
print err 
print " r2: " 
print r2

from sklearn import ensemble
bagged_dt = ensemble.BaggingClassifier(dt, n_estimators=5)

model_predictions = []
model_names = []

# Decision Tree
for i in [10,20,2,3,5]:
    dt = tree.DecisionTreeClassifier(max_depth=i)
    dt.fit(ftrain, ltrain)
    model_predictions.append(dt.predict_proba(ftest)[:,1])
    model_names.append('dt_' + str(i))
    
n_estimators=10

# Bagging + DT
for i in [5]:
    bg_dt_b = tree.DecisionTreeClassifier(max_depth=i)
    bg_dt = ensemble.BaggingClassifier(bg_dt_b, n_estimators=n_estimators, random_state=42)
    bg_dt.fit(ftrain, ltrain)
    model_predictions.append(bg_dt.predict_proba(ftest)[:,1])
    model_names.append('bagging_' + str(i) + '+dt')

import sklearn.metrics as ms
for i in range(len(model_predictions)):
    print model_names[i]
    print ms.roc_auc_score(ltest, model_predictions[i])



basic decision tree:
error: 
279004.799194
 r2: 
-0.640053695083
dt_10


ValueError: multiclass format is not supported

In [82]:
model_predictions

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.]),
 array([ 0.03571429,  0.        ,  0.03571429,  0.        ,  0.03571429,
         0.        ,  0.03571429,  0.03571429,  0.03571429,  0.03571429,
         0.        ,  0.03571429,  0.03571429,  0.03571429,  0.03571429,  0.        ]),
 array([ 0.04,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.04,  0.  ,  0.04,
         0.  ,  0.  ,  0.04,  0.  ,  0.  ,  0.  ,  0.  ]),
 array([ 0.05,  0.  ,  0.  ,  0.05,  0.  ,  0.05,  0.05,  0.  ,  0.05,
         0.  ,  0.  ,  0.05,  0.  ,  0.  ,  0.05,  0.05]),
 array([ 0.0228261 ,  0.        ,  0.02033693,  0.02509883,  0.01371857,
         0.02509883,  0.02509883,  0.02509883,  0.02509883,  0.01852453,
         0.02259883,  0.02509883,  0.01848047,  0.01371432,  0.02033693,
         0.02292492])]